In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import nltk
import re
import spacy
from gensim import corpora, models
import gensim
from gensim.matutils import hellinger
from scipy.spatial.distance import cosine
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
df = pd.read_csv(r'C:\Users\trevo\NLP\data\ReducedDataset.csv')

In [4]:
df.head(5)

,Unnamed: 0,ALink,SName,SLink,Lyric,language
0,139419,/foo-fighters/,"Hey, Johnny Park!",/foo-fighters/hey-johnny-park.html,Come and I'll take you under\nThis beautiful b...,en
1,290738,/mxpx/,Call In Sick,/mxpx/call-in-sick.html,"Oh how I missed you,\nOh how I needed you toda...",en
2,162905,/arch-enemy/,Despicable Heroes,/arch-enemy/despicable-heroes.html,"I spit in your face, preacers and leaders\nSpe...",en
3,281035,/the-maine/,Whoever She Is,/the-maine/whoever-she-is.html,I thought I had my girl but she ran away\nMy c...,en
4,253213,/a-ha/,Days On End,/a-ha/days-on-end.html,Do know why winter's such a cold and lonely pl...,en


In [6]:
df_en = df[df['language']=='en']

In [7]:
df_en.head(5)

,Unnamed: 0,ALink,SName,SLink,Lyric,language
0,139419,/foo-fighters/,"Hey, Johnny Park!",/foo-fighters/hey-johnny-park.html,Come and I'll take you under\nThis beautiful b...,en
1,290738,/mxpx/,Call In Sick,/mxpx/call-in-sick.html,"Oh how I missed you,\nOh how I needed you toda...",en
2,162905,/arch-enemy/,Despicable Heroes,/arch-enemy/despicable-heroes.html,"I spit in your face, preacers and leaders\nSpe...",en
3,281035,/the-maine/,Whoever She Is,/the-maine/whoever-she-is.html,I thought I had my girl but she ran away\nMy c...,en
4,253213,/a-ha/,Days On End,/a-ha/days-on-end.html,Do know why winter's such a cold and lonely pl...,en


In [8]:
df_en.shape[0]

38363

In [30]:
print(df_en['Lyric'].iloc[20])

I'm comin' home, I've done my time
Now I've got to know what is and isn't mine
If you received my letter telling you I'd soon be free
Then you'll know just what to do
If you still want me
If you still want me
Whoa, tie a yellow ribbon 'round the old oak tree
It's been three long years
Do ya still want me?
If I don't see a ribbon round the old oak tree
I'll stay on the bus
Forget about us
Put the blame on me
If I don't see a yellow ribbon round the old oak tree

Bus driver, please look for me
'cause I couldn't bear to see what I might see
I'm really still in prison
And my love, she holds the key
A simple yellow ribbon's what I need to set me free
I wrote and told her please

Whoa, tie a yellow ribbon round the old oak tree
It's been three long years
Do ya still want me?
If I don't see a ribbon round the old oak tree
I'll stay on the bus
Forget about us
Put the blame on me
If I don't see a yellow ribbon round the old oak tree

Now the whole damned bus is cheerin'
And I can't believe I se

In [33]:
# Download the stopwords library
nltk.download('stopwords')

# Establish a word punctuation tokenizer
wpt = nltk.WordPunctTokenizer()

# Establish the English stop words
basic_stop_words = nltk.corpus.stopwords.words('english')

custom_stop_words = ['get', 'yeah', 's', 'ai', 'ca', 'like', 'nt', 'ta', 'oh', 'got', 'gonna','goin','na','I', "i'm", "ain't", 'come', 'make', 'know', 'gotta']

stop_words = basic_stop_words + custom_stop_words

def normalize_document(doc):
    # Lowercase and remove special characters and whitespaces
    doc = re.sub(r"[^a-zA-Z\s']", '', doc, re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()

    # Tokenize document
    tokens = wpt.tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_words and not token.isdigit()]
    # Re-create the document from filtered tokens
    doc = ' '.join(filtered_tokens)

        # Remove punctuation
    doc = re.sub(f"[{re.escape(string.punctuation)}]", '', doc)

    doc = re.sub(r"'\s*", "", doc)
    return doc

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trevo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(df_en['Lyric'])

In [35]:
print(norm_corpus[0])

 take beautiful bruise  colors everything fades time  true wish another stab undercover change mind  impossible  let  never selling sit watch every mood eyes still remind angels hover eyes change blind blue  impossible  let  never selling sit watch every mood  found reward  throw away long  share piece mine  impossible  let  never selling sit watch every mood


In [37]:
doc = nlp(norm_corpus[0].item())

for token in doc:
    print(token.text, token.lemma_)

   
take take
beautiful beautiful
bruise bruise
   
colors color
everything everything
fades fade
time time
   
true true
wish wish
another another
stab stab
undercover undercover
change change
mind mind
   
impossible impossible
   
let let
   
never never
selling sell
sit sit
watch watch
every every
mood mood
eyes eye
still still
remind remind
angels angel
hover hover
eyes eye
change change
blind blind
blue blue
   
impossible impossible
   
let let
   
never never
selling sell
sit sit
watch watch
every every
mood mood
   
found find
reward reward
   
throw throw
away away
long long
   
share share
piece piece
mine mine
   
impossible impossible
   
let let
   
never never
selling sell
sit sit
watch watch
every every
mood mood


In [107]:
lemmatized_corpus = []

for text in norm_corpus:
    doc = nlp(text.item())  
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    lemmatized_corpus.append(lemmatized_text)

In [108]:
print(lemmatized_corpus[0])

  take beautiful bruise   color everything fade time   true wish another stab undercover change mind   impossible   let   never sell sit watch every mood eye still remind angel hover eye change blind blue   impossible   let   never sell sit watch every mood   find reward   throw away long   share piece mine   impossible   let   never sell sit watch every mood


In [109]:
df = pd.DataFrame(lemmatized_corpus, columns=['lyrics'])
df.to_csv('output_file.csv', index=False)

In [110]:
tokenized_corpus = [text.split() for text in lemmatized_corpus]

dictionary = corpora.Dictionary(tokenized_corpus)
corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]

In [111]:
# check the work frequency within our dictionary that we will be using for the lda model, if a word is too frequenct it may lose value
# Too frequent words have been added as stop words on each iteration.
top_10_most_frequent_words = sorted(dictionary.cfs.items(), key=lambda item: item[1], reverse=True)[:10]
print(top_10_most_frequent_words)
for token_id, frequency in top_10_most_frequent_words:
    word = dictionary[token_id]
    print(f"Word: {word}, Frequency: {frequency}")

[(202, 64544), (246, 54282), (209, 40490), (67, 37575), (32, 34549), (104, 33670), (16, 32458), (30, 31321), (282, 30233), (21, 29150)]
Word: love, Frequency: 64544
Word: go, Frequency: 54282
Word: say, Frequency: 40490
Word: see, Frequency: 37575
Word: time, Frequency: 34549
Word: one, Frequency: 33670
Word: let, Frequency: 32458
Word: take, Frequency: 31321
Word: want, Frequency: 30233
Word: never, Frequency: 29150


In [112]:
lda_model = gensim.models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=35, random_state = 42)

In [114]:
for topic_id in range(10):
    topic = lda_model.print_topic(topic_id)
    print(f"Topic {topic_id}: {topic}")

Topic 0: 0.119*"la" + 0.021*"boom" + 0.020*"joan" + 0.019*"baez" + 0.019*"beautiful" + 0.018*"bye" + 0.017*"greedy" + 0.017*"hello" + 0.017*"de" + 0.015*"e"
Topic 1: 0.060*"baby" + 0.041*"girl" + 0.030*"wanna" + 0.023*"want" + 0.021*"say" + 0.020*"hey" + 0.019*"love" + 0.014*"ooh" + 0.014*"boy" + 0.013*"right"
Topic 2: 0.045*"love" + 0.021*"go" + 0.019*"time" + 0.018*"never" + 0.017*"say" + 0.015*"one" + 0.014*"feel" + 0.014*"see" + 0.012*"heart" + 0.012*"want"
Topic 3: 0.111*"lyric" + 0.063*"bill" + 0.053*"monroe" + 0.038*"blue" + 0.020*"woah" + 0.019*"lyrics" + 0.016*"new" + 0.011*"city" + 0.010*"train" + 0.010*"john"
Topic 4: 0.021*"man" + 0.017*"well" + 0.016*"go" + 0.014*"one" + 0.014*"say" + 0.011*"run" + 0.010*"people" + 0.009*"old" + 0.009*"look" + 0.007*"see"
Topic 5: 0.014*"nigga" + 0.014*"shit" + 0.013*"fuck" + 0.012*"bitch" + 0.011*"ya" + 0.009*"they" + 0.009*"niggas" + 0.009*"yo" + 0.008*"money" + 0.008*"back"
Topic 6: 0.018*"life" + 0.013*"take" + 0.012*"feel" + 0.011*"di

In [115]:
# Verify that a singular word is not used in a large majority of the topics or else that word may not hold much value
word_counts = {}
num_topics = lda_model.num_topics

for topic_id in range(num_topics):
    topic_words = lda_model.show_topic(topic_id, topn=10) 

    for word, prob in topic_words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

word_count_topic_list = pd.DataFrame(list(word_counts.items()), columns=['Word', 'Count'])

word_count_topic_list = word_count_topic_list.sort_values(by='Count', ascending=False)

print(word_count_topic_list.head(10))

       Word  Count
14      say      3
25      see      3
20       go      3
24     feel      2
23      one      2
65    night      2
16     love      2
13     want      2
56      lie      1
63  tonight      1


In [116]:
song_topic_distribution = [lda_model[doc] for doc in corpus]

In [117]:
def calculate_hellinger_distance(song_dist_1, song_dist_2):
    return hellinger(song_dist_1, song_dist_2)

input_song_index = 139

input_song_dist = song_topic_distribution[input_song_index]

hellinger_distances = []

for i, song_dist in enumerate(song_topic_distribution):

    if i != input_song_index:
        
        distance = calculate_hellinger_distance(input_song_dist, song_dist)
        hellinger_distances.append((distance, i))

hellinger_distances.sort(key=lambda x: x[0])

top_5_similar_songs = hellinger_distances[:5]

# Print the results
print(f"\nTop 5 most similar songs to the input song {df_en['SName'].iloc[input_song_index]} by {df_en['ALink'].iloc[input_song_index]}:")
for distance, song_index in top_5_similar_songs:
    print(f"Song {df_en['SName'].iloc[song_index]} by {df_en['ALink'].iloc[song_index]}")


Top 5 most similar songs to the input song Star by /beck/:
Song See Water by /beck/
Song Sober And Unkissed by /sia/
Song Sausalito by /bright-eyes/
Song The end by /alphaville/
Song No Snow On The Mountain by /nada-surf/


In [70]:
song_topic_distribution[65]

[(2, 0.43475226),
 (3, 0.28249836),
 (11, 0.12165525),
 (13, 0.07855593),
 (14, 0.07209814)]

In [77]:
print(df_en['SName'].iloc[65])
print(df_en['Lyric'].iloc[65])

So Much Pain
[Master P and (Mo B. Dick)]
Oh like that (Ooh)
Tre-8 they ain't ready for this dog (Ooh)
Smoke One and No Limit (Ooh)
All the way from California to New Orleans
Ask em' about it, so much pain boy (So much pain)

[Master P]
Birds in the kitchen, palms itchin'
And all y'all niggas in the game pay attention
As I teach, ain't got no time to preach
2 for 3, 4 for 5, 16 a fuckin' key
Don't laugh, niggas like to backstab
But where I'm from see yo brother on a body slab
New Orleans, the city of the candy cream
A bunch of projects full of jackers and dope fiends
As I cry, think one day I gotta die
But I don't give a fuck cause ain't no love from the outside
As I walk to the projects
Niggas killin' dope fiends behind fuckin' county checks
And my younger homie smokin' dope
The niggas I used to hang with doin' that boy broke
And they gone off that water, water
Ain't no love from New Orleans all the way back to Florida
It's just a bunch of pain

[Chorus: Mo B. Dick]
So much pain, so mu

In [75]:
song_topic_distribution[2881]

[(0, 0.026852738),
 (1, 0.5015815),
 (2, 0.075799875),
 (3, 0.10675562),
 (6, 0.023035636),
 (7, 0.019402707),
 (11, 0.13886228),
 (12, 0.090286404)]

In [76]:
print(df_en['SName'].iloc[2881])
print(df_en['Lyric'].iloc[2881])

Run This Town
(Rihanna)
Feel it coming in the air
Hear the screams from everywhere
I'm addicted to the the thrill
It's a dangerous love affair
Can't be scaring nickels down
Got a problem, tell me now
Only thing that's on my mind
Is who gon' run this town tonight
Is who gon' run this town tonight
We gon' run this town

(Jay-Z)
We are, yeah, I said it, we are
This is Roc Nation, pledge your allegiance
Get y'all fatigues on, all black everything
Black cards, black cars, all black everything
And our girls are blackbirds, riding with they Dillingers
I get more in-depth if you boys really real enough
This is La Familia, I'll explain later
But for now, let me get back to this paper
I'm a couple bands down and I'm tryna get back
I gave Doug a grip, I lost a flip for five stacks
Yeah, I'm talking five comma six zeroes dot zero, here girl
Back to running circles 'round niggas, now we squared up
Hold up

(Rihanna)
Life's a game but it's not fair
I break the rules so I don't care
So I keep doing m

In [85]:
print(df_en['Lyric'].iloc[52])

Time flies by when the night is young
Daylight shines on an undisclosed location,
location
Bloodshot eyes looking for the sun
Paradise delivered and
we call it a vacation, vacation

You're painting me a dream that I
Wanna belong, yeah, wanna belong, yeah

Over the hills and far away
A million miles from L.A
Just anywhere away with you
I know we've got to get away
Someplace where no one knows
our name
We'll find the start to something new
Just take me anywhere,
take me anywhere
Anywhere away with you
Just take me anywhere,
take me anywhere
Anywhere away with you

Fun, little less fun
Little less, over, over,
over, over, me
Oh, fun, little less fun
Little less, over, over,
over, over, me

Truth comes out when we're blacking out
Looking for connection
in a crowd of empty faces,
empty faces
Your secrets are the only
thing I'm craving now
The good, and the bad, let me in
'Cause I can take it, I can take it

You're painting me a dream that I
Wanna belong, yeah, wanna belong, yeah

Over the h

In [86]:
print(df_en['Lyric'].iloc[1427])

I can't survive
Living without you baby
Deep in the night
I feel so cold inside

We used to say
"We'll be as one forever"
No matter where you are
I'll be loving you

So far away
You're so far away from me
I can't live without you baby
So far away
You're so far away from me


In [110]:
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_score(lyrics):
    sentiment = analyzer.polarity_scores(lyrics)
    return sentiment['neg']

sentiment1 = get_sentiment_score(lemmatized_corpus[496])
sentiment2 = get_sentiment_score(lemmatized_corpus[274])

tfidf_vectorizer = TfidfVectorizer()
lyrics_matrix = tfidf_vectorizer.fit_transform([lemmatized_corpus[496], lemmatized_corpus[274]])
lyrics_similarity = cosine_similarity(lyrics_matrix)

print("Sentiment Score for Song 1:", sentiment1)
print("Sentiment Score for Song 2:", sentiment2)
print("Cosine Similarity between Lyrics:", lyrics_similarity[0][1])

Sentiment Score for Song 1: 0.171
Sentiment Score for Song 2: 0.378
Cosine Similarity between Lyrics: 0.17731845777226657


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\trevo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
## Preprocessing
# Tokenize
# Lowercase
# punctuation removal
# number removal
# Stop word removal

## Additional Preprocessing
# Lemmatization and or stemming
# Analysis word frequency determine if we need to remove high frequency low value words
# Stop word removal

## Data Check
# Explore word stats
# Everything is clean

## NLP Processing
# Sentiment Analysis
# LDA
# POS taging/Similarity

## Front End
# Django or Flask additions

Test